In [3]:
import numpy as np
np.random.seed(0)

# model(X_test, training = True)
array = lambda: np.random.rand(10, 1)
print(array().shape)
Y_test = np.array([1,0,1,1,0,1,1,1,0,0])
print("Y_test", Y_test.shape)

preds = np.array([array() for _ in range(5)])
print(preds.shape)
preds = preds.mean(axis=0)
print(preds.shape)
print(preds)
Y_pred = np.round(preds)
print(Y_pred)
test_accuracy = np.mean(Y_pred == Y_test)
# test_accuracy = np.sum(Y_pred == Y_test) / len(Y_test)
# test_accuracy_list.append(test_accuracy)
test_accuracy

(10, 1)
Y_test (10,)
(5, 10, 1)
(10, 1)
[[0.59292073]
 [0.59558413]
 [0.63433586]
 [0.48736601]
 [0.21674875]
 [0.43532664]
 [0.32783166]
 [0.55528813]
 [0.60509845]
 [0.51492615]]
[[1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


0.52